<a href="https://colab.research.google.com/github/gabopy-glitch/Aprendizaje-Autom-tico/blob/main/01_Mi_segundo_modelo_Piedra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/01-Mi-segundo-modelo-Plantilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Facultad de Ciencias Exactas, Naturales y Ambientales</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Mi primer modelo
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Nombre y Apellido &bull; Fecha
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

Este notebook está diseñado parcticar en el entrenamiento y evaluación de un modelo de regresión lineal utilizando datos reales. Los objetivos principales son:

- Entrenar un modelo de regresión lineal.
- Evaluar el ajuste del modelo.


Del conjunto de datos disponible [aquí](https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv), se desea predecir la variable `price` en función de las variables `bedrooms`, `bathrooms`, `sqft_living`, `sqft_lot`, `floors`.

In [100]:
#Librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

---
## <font color='264CC7'> Lectura de Datos </font>


Realiza la lectura de los datos y presenta las primeras filas del conjunto de datos.

In [101]:
url = "https://raw.githubusercontent.com/mGalarnyk/Tutorial_Data/master/King_County/kingCountyHouseData.csv"
df = pd.read_csv(url)
df.head()

#Reducción a las colmnas solicitadas
Y = pd.DataFrame(df['price'])
X = df[['bathrooms', 'bedrooms', 'sqft_living', 'sqft_lot', 'floors']]

df = pd.concat([Y, X], axis=1)
df.head()

,price,bathrooms,bedrooms,sqft_living,sqft_lot,floors
0,221900.0,1.00,3,1180,5650,1.0
1,538000.0,2.25,3,2570,7242,2.0
2,180000.0,1.00,2,770,10000,1.0
3,604000.0,3.00,4,1960,5000,1.0
4,510000.0,2.00,3,1680,8080,1.0


In [102]:
#Estadistica descriptiva para mejor entendimineto
df.describe()

,price,bathrooms,bedrooms,sqft_living,sqft_lot,floors
count,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000
mean,5.400881e+05,2.114757,3.370842,2079.899736,1.510697e+04,1.494309
std,3.671272e+05,0.770163,0.930062,918.440897,4.142051e+04,0.539989
min,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000
25%,3.219500e+05,1.750000,3.000000,1427.000000,5.040000e+03,1.000000
50%,4.500000e+05,2.250000,3.000000,1910.000000,7.618000e+03,1.500000
75%,6.450000e+05,2.500000,4.000000,2550.000000,1.068800e+04,2.000000
max,7.700000e+06,8.000000,33.000000,13540.000000,1.651359e+06,3.500000


---
## <font color='264CC7'> Preprocesamiento de datos </font>

Realiza una normalización de los datos, prueba diferentes técnicas y selecciona la que mejor se ajuste a los datos.

In [103]:
#Aplicamos diferentes procesos de normalización

#Máximos
x_max = X.max()
X_max_normalized = X / x_max
print("Normalización por el máximo: \n")
display(X_max_normalized.head())

#Min-Max
x_min_max = MinMaxScaler()
X_min_max_normalized = pd.DataFrame(x_min_max.fit_transform(X), columns=X.columns)
print("Normalización min_max: \n")
display(X_min_max_normalized.head())

#Standar Scaler
standar_scaler = StandardScaler()
X_standar_normalized = pd.DataFrame(standar_scaler.fit_transform(X), columns=X.columns)
print("Normalización standar: \n")
display(X_standar_normalized.head())

Normalización por el máximo: 



,bathrooms,bedrooms,sqft_living,sqft_lot,floors
0,0.12500,0.090909,0.087149,0.003421,0.285714
1,0.28125,0.090909,0.189808,0.004385,0.571429
2,0.12500,0.060606,0.056869,0.006056,0.285714
3,0.37500,0.121212,0.144756,0.003028,0.285714
4,0.25000,0.090909,0.124077,0.004893,0.285714


Normalización min_max: 



,bathrooms,bedrooms,sqft_living,sqft_lot,floors
0,0.12500,0.090909,0.067170,0.003108,0.0
1,0.28125,0.090909,0.172075,0.004072,0.4
2,0.12500,0.060606,0.036226,0.005743,0.0
3,0.37500,0.121212,0.126038,0.002714,0.0
4,0.25000,0.090909,0.104906,0.004579,0.0


Normalización standar: 



,bathrooms,bedrooms,sqft_living,sqft_lot,floors
0,-1.447464,-0.398737,-0.979835,-0.228321,-0.915427
1,0.175607,-0.398737,0.533634,-0.189885,0.936506
2,-1.447464,-1.473959,-1.426254,-0.123298,-0.915427
3,1.149449,0.676485,-0.130550,-0.244014,-0.915427
4,-0.149007,-0.398737,-0.435422,-0.169653,-0.915427


In [104]:
# Aqui aplicaremos las normalizaciones no vistas en calse
# MaxAbs Scaler
x_max_abs = MaxAbsScaler()
X_max_abs_normalized = pd.DataFrame(x_max_abs.fit_transform(X), columns=X.columns)
print('Normalizazión MaxAbs: \n')
display(X_max_abs_normalized)

# Robust Scaler
x_robust = RobustScaler()
X_robust_normalized = pd.DataFrame(x_robust.fit_transform(X), columns=X.columns)
print('Normalizazión Robust: \n')
display(X_robust_normalized)

# Normalizer
x_normalizer = Normalizer()
X_normalizer_normalized = pd.DataFrame(x_normalizer.fit_transform(X), columns=X.columns)
print('Normalizazión Normalizer: \n')
display(X_normalizer_normalized)

# Quantile Transformer
x_quantile = QuantileTransformer()
X_quantile_normalized = pd.DataFrame(x_quantile.fit_transform(X), columns=X.columns)
print('Normalizazión Quantile: \n')
display(X_quantile_normalized)

# Power Transformer
x_power = PowerTransformer()
X_power_normalized = pd.DataFrame(x_power.fit_transform(X), columns=X.columns)
print('Normalizazión Power: \n')
display(X_power_normalized)

Normalizazión MaxAbs: 



,bathrooms,bedrooms,sqft_living,sqft_lot,floors
0,0.12500,0.090909,0.087149,0.003421,0.285714
1,0.28125,0.090909,0.189808,0.004385,0.571429
2,0.12500,0.060606,0.056869,0.006056,0.285714
3,0.37500,0.121212,0.144756,0.003028,0.285714
4,0.25000,0.090909,0.124077,0.004893,0.285714
...,...,...,...,...,...
21608,0.31250,0.090909,0.112999,0.000685,0.857143
21609,0.31250,0.121212,0.170606,0.003520,0.571429
21610,0.09375,0.060606,0.075332,0.000818,0.571429
21611,0.31250,0.090909,0.118168,0.001446,0.571429


Normalizazión Robust: 



,bathrooms,bedrooms,sqft_living,sqft_lot,floors
0,-1.666667,0.0,-0.650045,-0.348442,-0.5
1,0.000000,0.0,0.587711,-0.066572,0.5
2,-1.666667,-1.0,-1.015138,0.421742,-0.5
3,1.000000,1.0,0.044524,-0.463527,-0.5
4,-0.333333,0.0,-0.204809,0.081799,-0.5
...,...,...,...,...,...
21608,0.333333,0.0,-0.338379,-1.148548,1.5
21609,0.333333,1.0,0.356189,-0.319582,0.5
21610,-2.000000,-1.0,-0.792520,-1.109773,0.5
21611,0.333333,0.0,-0.276046,-0.925992,0.5


Normalizazión Normalizer: 



,bathrooms,bedrooms,sqft_living,sqft_lot,floors
0,0.000173,0.000520,0.204438,0.978879,0.000173
1,0.000293,0.000390,0.334440,0.942417,0.000260
2,0.000100,0.000199,0.076773,0.997049,0.000100
3,0.000559,0.000745,0.364961,0.931022,0.000186
4,0.000242,0.000364,0.203567,0.979061,0.000121
...,...,...,...,...,...
21608,0.001314,0.001577,0.804141,0.594433,0.001577
21609,0.000400,0.000639,0.369295,0.929312,0.000320
21610,0.000443,0.001182,0.602832,0.797866,0.001182
21611,0.000870,0.001044,0.556625,0.830763,0.000696


Normalizazión Quantile: 



,bathrooms,bedrooms,sqft_living,sqft_lot,floors
0,0.093594,0.365866,0.132132,0.309309,0.000000
1,0.530030,0.365866,0.757758,0.457685,0.771271
2,0.093594,0.073073,0.015516,0.717217,0.000000
3,0.896897,0.753253,0.524024,0.239239,0.000000
4,0.437437,0.365866,0.386887,0.555730,0.000000
...,...,...,...,...,...
21608,0.701201,0.365866,0.308308,0.015722,0.984985
21609,0.701201,0.753253,0.672673,0.323270,0.771271
21610,0.002503,0.073073,0.077077,0.032330,0.771271
21611,0.701201,0.365866,0.344344,0.064131,0.771271


Normalizazión Power: 



,bathrooms,bedrooms,sqft_living,sqft_lot,floors
0,-1.571853,-0.352407,-1.121655,-0.331682,-0.975610
1,0.250882,-0.352407,0.706456,-0.032289,1.029432
2,-1.571853,-1.605435,-2.107060,0.336051,-0.975610
3,1.136536,0.722633,0.065492,-0.484436,-0.975610
4,-0.072697,-0.352407,-0.296964,0.095276,-0.975610
...,...,...,...,...,...
21608,0.559169,-0.352407,-0.516100,-2.661036,1.876834
21609,0.559169,0.722633,0.453567,-0.296649,1.029432
21610,-2.018163,-1.605435,-1.459381,-2.368001,1.029432
21611,0.559169,-0.352407,-0.411354,-1.488860,1.029432


---
## <font color='264CC7'> Características y etiquetas </font>

Separa las características y las etiquetas y mostrar la dimensión de cada conjunto.

In [105]:
# Dividir el conjunto de datos en características y etiquetas
features = ['bathrooms', 'bedrooms', 'sqft_living', 'sqft_lot', 'floors']
X = df.loc[:, features]
y = df.loc[:, ['price']]

# Aplicamos lo mismo a las normalizaciones para plicarlas al modelo
X_max = X_max_normalized.loc[:, features]
X_min_max = X_min_max_normalized.loc[:, features]
X_standar = X_standar_normalized.loc[:, features]

In [106]:
#Dividimos para las nuevas normalizaciones
X_max_abs = X_max_abs_normalized.loc[:, features]
X_robust = X_robust_normalized.loc[:, features]
X_normalizer = X_normalizer_normalized.loc[:, features]
X_quantile = X_quantile_normalized.loc[:, features]
X_power = X_power_normalized.loc[:, features]

---
## <font color='264CC7'> Conjunto de entrenamiento y prueba </font>

Divide los datos en dos conjuntos: uno para entrenamiento y otro para prueba y visualiza la dimensión de cada conjunto. Usar `random_state` igual al últmo número de tu cédula.

In [107]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

#De igual forma dividimos nuestros nuestros datos normalizados
X_train_max, X_test_max, y_train_max, y_test_max = train_test_split(X_max, y, test_size=0.2, random_state=6)
X_train_min_max, X_test_min_max, y_train_min_max, y_test_min_max = train_test_split(X_min_max, y, test_size=0.2, random_state=6)
X_train_standar, X_test_standar, y_train_standar, y_test_standar = train_test_split(X_standar, y, test_size=0.2, random_state=6)

In [108]:
# Entrenamineto y prueba para nuevas normalizaciones
X_train_max_abs, X_test_max_abs, y_train_max_abs, y_test_max_abs = train_test_split(X_max_abs, y, test_size=0.2, random_state=6)
X_train_robust, X_test_robust, y_train_robust, y_test_robust = train_test_split(X_robust, y, test_size=0.2, random_state=6)
X_train_normalizer, X_test_normalizer, y_train_normalizer, y_test_normalizer = train_test_split(X_normalizer, y, test_size=0.2, random_state=6)
X_train_quantile, X_test_quantile, y_train_quantile, y_test_quantile = train_test_split(X_quantile, y, test_size=0.2, random_state=6)
X_train_power, X_test_power, y_train_power, y_test_power = train_test_split(X_power, y, test_size=0.2, random_state=6)

Revisa la distribución de las etiquetas en los conjuntos de entrenamiento y prueba.

In [109]:
# Reviso las dimensiones de los conjuntos de datos
print('Dimensiones de los conjuntos de datos:')
print('X_train:', X_train.shape)
print('y_train:', y_train.shape)
print('X_test:', X_test.shape)
print('y_test:', y_test.shape)

Dimensiones de los conjuntos de datos:
X_train: (17290, 5)
y_train: (17290, 1)
X_test: (4323, 5)
y_test: (4323, 1)


---
## <font color='264CC7'> Entrenamiento </font>

Define un modelo de regresión lineal y lo entrena con los datos de entrenamiento.

In [110]:
# Crear el modelo y ajustarlo
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Visualizar los coeficientes
print('Ecuación de la recta:')
print('y =', modelo.coef_[0][0], 'x +', modelo.intercept_[0])

# Creamos los modelos para nuestras normalizaciones
modelo_max = LinearRegression()
modelo_max.fit(X_train_max, y_train_max)
modelo_min_max = LinearRegression()
modelo_min_max.fit(X_train_min_max, y_train_min_max)
modelo_standar = LinearRegression()
modelo_standar.fit(X_train_standar, y_train_standar)

# Visualizamos sus ecuaciones
print('Ecuación de la recta Maximo:')
print('y =', modelo_max.coef_[0][0], 'x +', modelo_max.intercept_[0])
print('Ecuación de la recta Min_Max:')
print('y =', modelo_min_max.coef_[0][0], 'x +', modelo_min_max.intercept_[0])
print('Ecuación de la recta Standar:')
print('y =', modelo_standar.coef_[0][0], 'x +', modelo_standar.intercept_[0])

Ecuación de la recta:
y = 2941.984306474792 x + 94115.86941033084
Ecuación de la recta Maximo:
y = 23535.87445089249 x + 94115.86941047374
Ecuación de la recta Min_Max:
y = 23535.874450895317 x + 186558.26163699199
Ecuación de la recta Standar:
y = 2265.7555035567602 x + 541032.8848969664


In [111]:
# Modelos de las nuevas normalizaciones
modelo_max_abs = LinearRegression()
modelo_max_abs.fit(X_train_max_abs, y_train_max_abs)
modelo_robust = LinearRegression()
modelo_robust.fit(X_train_robust, y_train_robust)
modelo_normalizer = LinearRegression()
modelo_normalizer.fit(X_train_normalizer, y_train_normalizer)
modelo_quantile = LinearRegression()
modelo_quantile.fit(X_train_quantile, y_train_quantile)
modelo_power = LinearRegression()
modelo_power.fit(X_train_power, y_train_power)

# Y vermos sus ecuaciones
print('Ecuacion de la recta de MaxAbs:')
print('y =', modelo_max_abs.coef_[0][0], 'x +', modelo_max_abs.intercept_[0])
print('Ecuacion de la recta de Robust:')
print('y =', modelo_robust.coef_[0][0], 'x +', modelo_robust.intercept_[0])
print('Ecuacion de la recta de Normalizer:')
print('y =', modelo_normalizer.coef_[0][0], 'x +', modelo_normalizer.intercept_[0])
print('Ecuacion de la recta de Quantile')
print('y =', modelo_quantile.coef_[0][0], 'x +', modelo_quantile.intercept_[0])
print('Ecuacion de la recta de Power:')
print('y =', modelo_power.coef_[0][0], 'x +', modelo_power.intercept_[0])

Ecuacion de la recta de MaxAbs:
y = 23535.87445089249 x + 94115.86941047374
Ecuacion de la recta de Robust:
y = 2206.488229771487 x + 513872.10547565675
Ecuacion de la recta de Normalizer:
y = -203585923.78016514 x + 1220319.0678885628
Ecuacion de la recta de Quantile
y = 71766.90881692116 x + 191560.266177168
Ecuacion de la recta de Power:
y = 20749.05685366719 x + 541137.8274973043


---
## <font color='264CC7'> Evaluacion </font>

Realiza la predicciones con el conjunto de prueba y muestra los valores en una tabla.

In [112]:
# Predicciones del modelo
y_pred = modelo.predict(X_test)

# Comparar las predicciones con los datos de prueba
df_predicciones = pd.DataFrame({
    'Actual': y_test.values.flatten(),
    'Predicción': y_pred.flatten()
})
display(df_predicciones)

# Hacemos predicciones para x normalizada
y_pred_max = modelo_max.predict(X_test_max)
y_pred_min_max = modelo_min_max.predict(X_test_min_max)
y_pred_standar = modelo_standar.predict(X_test_standar)

# Comparamos las predicciones a las pruebas
df_predicciones_max = pd.DataFrame({
    'Actual': y_test_max.values.flatten(),
    'Predicción': y_pred_max.flatten()
})

df_predicciones_min_max = pd.DataFrame({
    'Actual': y_test_min_max.values.flatten(),
    'Predicción': y_pred_min_max.flatten()
})

df_predicciones_standar = pd.DataFrame({
    'Actual': y_test_standar.values.flatten(),
    'Predicción': y_pred_standar.flatten()
})

display(df_predicciones_max)
display(df_predicciones_min_max)
display(df_predicciones_standar)

,Actual,Predicción
0,380000.0,731866.093800
1,309000.0,569129.540835
2,500000.0,420893.021155
3,233000.0,532395.593993
4,565000.0,883330.633962
...,...,...
4318,490000.0,467726.739039
4319,640000.0,651408.171691
4320,450000.0,333369.173327
4321,495000.0,377211.544135


,Actual,Predicción
0,380000.0,731866.093801
1,309000.0,569129.540835
2,500000.0,420893.021155
3,233000.0,532395.593994
4,565000.0,883330.633962
...,...,...
4318,490000.0,467726.739039
4319,640000.0,651408.171691
4320,450000.0,333369.173327
4321,495000.0,377211.544135


,Actual,Predicción
0,380000.0,731866.093801
1,309000.0,569129.540835
2,500000.0,420893.021155
3,233000.0,532395.593994
4,565000.0,883330.633962
...,...,...
4318,490000.0,467726.739039
4319,640000.0,651408.171691
4320,450000.0,333369.173327
4321,495000.0,377211.544135


,Actual,Predicción
0,380000.0,731866.093801
1,309000.0,569129.540835
2,500000.0,420893.021155
3,233000.0,532395.593994
4,565000.0,883330.633962
...,...,...
4318,490000.0,467726.739039
4319,640000.0,651408.171691
4320,450000.0,333369.173327
4321,495000.0,377211.544135


In [113]:
# Ahora prediciomos con las nuevas normalizaciones
y_pred_max_abs = modelo_max_abs.predict(X_test_max_abs)
y_pred_robust = modelo_robust.predict(X_test_robust)
y_pred_normalizer = modelo_normalizer.predict(X_test_normalizer)
y_pred_quantile = modelo_quantile.predict(X_test_quantile)
y_pred_power = modelo_power.predict(X_test_power)

# Predicciones
df_predicciones_max_abs = pd.DataFrame({
    'Actual': y_test_max_abs.values.flatten(),
    'Predicción': y_pred_max_abs.flatten()
})

df_predicciones_robust = pd.DataFrame({
    'Actual': y_test_robust.values.flatten(),
    'Predicción': y_pred_robust.flatten()
})

df_predicciones_normalizer = pd.DataFrame({
    'Actual': y_test_normalizer.values.flatten(),
    'Prediccion': y_pred_normalizer.flatten()
})

df_predicciones_quantile = pd.DataFrame({
    'Actual': y_test_quantile.values.flatten(),
    'Prediccion': y_pred_quantile.flatten()
})

df_predicciones_power = pd.DataFrame({
    'Actual': y_test_power.values.flatten(),
    'Prediccion': y_pred_power.flatten()
})

display(df_predicciones_max_abs)
display(df_predicciones_robust)
display(df_predicciones_normalizer)
display(df_predicciones_quantile)
display(df_predicciones_power)

,Actual,Predicción
0,380000.0,731866.093801
1,309000.0,569129.540835
2,500000.0,420893.021155
3,233000.0,532395.593994
4,565000.0,883330.633962
...,...,...
4318,490000.0,467726.739039
4319,640000.0,651408.171691
4320,450000.0,333369.173327
4321,495000.0,377211.544135


,Actual,Predicción
0,380000.0,731866.093801
1,309000.0,569129.540835
2,500000.0,420893.021155
3,233000.0,532395.593994
4,565000.0,883330.633962
...,...,...
4318,490000.0,467726.739039
4319,640000.0,651408.171691
4320,450000.0,333369.173327
4321,495000.0,377211.544135


,Actual,Prediccion
0,380000.0,725526.060372
1,309000.0,523360.745479
2,500000.0,548146.041979
3,233000.0,600838.202438
4,565000.0,925948.823192
...,...,...
4318,490000.0,544824.027707
4319,640000.0,869271.918049
4320,450000.0,433572.202114
4321,495000.0,565791.173990


,Actual,Prediccion
0,380000.0,715812.134030
1,309000.0,690552.293219
2,500000.0,425670.870474
3,233000.0,538175.915405
4,565000.0,837466.705360
...,...,...
4318,490000.0,481381.973019
4319,640000.0,662565.984474
4320,450000.0,323144.633172
4321,495000.0,334682.692807


,Actual,Prediccion
0,380000.0,718714.411887
1,309000.0,627599.162718
2,500000.0,472477.807705
3,233000.0,554234.046386
4,565000.0,843441.271732
...,...,...
4318,490000.0,513584.293268
4319,640000.0,692508.626933
4320,450000.0,353340.712604
4321,495000.0,405774.589607


Calcula el error cuadrático medio.

In [114]:
# Métricas de evaluación
mse = mean_squared_error(y_test, y_pred)

print('Error cuadrático medio:', mse)

# Métricas con normalización
mse_max = mean_squared_error(y_test_max, y_pred_max)
mse_min_max = mean_squared_error(y_test_min_max, y_pred_min_max)
mse_standar = mean_squared_error(y_test_standar, y_pred_standar)

print('Error cuadrático medio Maximo:', mse_max)
print('Error cuadrático medio Min_Max:', mse_min_max)
print('Error cuadrático medio Standar:', mse_standar)

Error cuadrático medio: 66046463199.98638
Error cuadrático medio Maximo: 66046463199.987724
Error cuadrático medio Min_Max: 66046463199.98771
Error cuadrático medio Standar: 66046463199.98771


In [115]:
# Métricas de evalucion con nuevas normalizaciones
mse_max_abs = mean_squared_error(y_test_max_abs, y_pred_max_abs)
mse_robust = mean_squared_error(y_test_robust, y_pred_robust)
mse_normalizer = mean_squared_error(y_test_normalizer, y_pred_normalizer)
mse_quantile = mean_squared_error(y_test_quantile, y_pred_quantile)
mse_power = mean_squared_error(y_test_power, y_pred_power)

#Imprimimos los resultados
print('Error cuadrático medio MaxAbs:', mse_max_abs)
print('Error cuadrático medio Robust:', mse_robust)
print('Error cuadrático medio Normalizer:', mse_normalizer)
print('Error cuadrático medio Quantile:', mse_quantile)
print('Error cuadrático medio Power:', mse_power)

Error cuadrático medio MaxAbs: 66046463199.987724
Error cuadrático medio Robust: 66046463199.98771
Error cuadrático medio Normalizer: 109151455296.15385
Error cuadrático medio Quantile: 90211266027.13707
Error cuadrático medio Power: 80628906195.4419


Escribe el error cuadrático medio para cada tipo de normalización:

- Normalización MaxAbsScaler:        66046463199.99
- Normalización MinMaxScaler:        66046463199.99
- Normalización RobustScaler:        66046463199.99
- Normalización StandardScaler:      66046463199.99
- Normalización Normalizer:          109151455296.15
- Normalización QuantileTransformer: 89835435963.11
- Normalización PowerTransformer:    80628906195.44

Escribe un párrafo sobre qué hace normalización no vista en clase:

- **Normalización Robust Scaler:** Elimina la mediana y escala los datos según el rango intercuartílico (entre el primer y tercer cuartil). Toma cada carácterística del modelo, la centra y la escala.

- **Normalización Normalizer:** Escala cada fila de la matriz a su norma, ya sea I1, I2 o inf siempre que tenga al menos un componente diferente de cero

- **Normalización Quantile Transformer:** Se aplica a cada característica del conjunto. se basa en estimar la distribución de probabilidad acumulada de cada característica para luego asignar cada valor original a una distribución uniforme, lUego se asignan a una distribución de salida obtenida por medio de la función de cuantiles asociada.

- **Normalización Power Transformer:** SOn una familia de transpormaciones paramétricas y monótonas que se usa para aproximar los datos a una gaussiana. De froma predeterminada se aplica normalización de varianza unitaria y media cero para todos los datos

---
## <font color='264CC7'> Cross Validation </font>

Realiza validación cruzada con 5 particiones y muestra el error cuadrático medio promedio con la normalización que mejor se ajuste a los datos para confirmar que el modelo no está sobreajustado.

In [116]:
# Definimos las particiones con KFold
kf = KFold(n_splits=5, shuffle=True, random_state=6)

scores_max = cross_val_score(
    modelo_max,
    X_max_abs_normalized,
    y,
    scoring='neg_mean_squared_error',
    cv=kf
)

mse_max = -scores_max.mean()

scores_min_max = cross_val_score(
    modelo_min_max,
    X_min_max_normalized,
    y,
    scoring='neg_mean_squared_error',
    cv=kf
)

mse_min_max = -scores_min_max.mean()

scores_standar = cross_val_score(
    modelo_standar,
    X_standar_normalized,
    y,
    scoring='neg_mean_squared_error',
    cv=kf
)

mse_standar = -scores_standar.mean()

scores_robust = cross_val_score(
    modelo_robust,
    X_robust_normalized,
    y,
    scoring='neg_mean_squared_error',
    cv=kf
)

mse_robust = -scores_robust.mean()

scores_normalizer = cross_val_score(
    modelo_normalizer,
    X_normalizer_normalized,
    y,
    scoring='neg_mean_squared_error',
    cv=kf
)

mse_normalizer = -scores_normalizer.mean()

scores_quantile = cross_val_score(
    modelo_quantile,
    X_quantile_normalized,
    y,
    scoring='neg_mean_squared_error',
    cv=kf
)

mse_quantile = -scores_quantile.mean()

scores_power = cross_val_score(
    modelo_power,
    X_power_normalized,
    y,
    scoring='neg_mean_squared_error',
    cv=kf
)

mse_power = -scores_power.mean()

# Imprimimos los resultados
print('Error cuadrático medio MaxAbs:', mse_max)
print()
print('Error cuadrático medio Min Max:', mse_min_max)
print()
print('Error cuadrático medio Standar:', mse_standar)
print()
print('Error cuadrático medio Robust:', mse_robust)
print()
print('Error cuadrático medio Normalizer:', mse_normalizer)
print()
print('Error cuadrático medio Quantile:', mse_quantile)
print()
print('Error cuadrático medio Power:', mse_power)

Error cuadrático medio MaxAbs: 66351816154.84999

Error cuadrático medio Min Max: 66351816154.84999

Error cuadrático medio Standar: 66351816154.84999

Error cuadrático medio Robust: 66351816154.84999

Error cuadrático medio Normalizer: 106714100189.87782

Error cuadrático medio Quantile: 90694726144.94373

Error cuadrático medio Power: 81699731383.81409
